# CV Routine

In [1]:
%matplotlib inline
# Load all necessary packages
import sys

sys.path.append("../")
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.fairness_adjuster import FairnessAdjuster
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import (
    load_preproc_data_adult,
    load_preproc_data_compas,
    load_preproc_data_german,
)
from aif360.datasets import (
    AdultDataset,
    BinaryLabelDataset,
    CompasDataset,
    GermanDataset,
)
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from IPython.display import Markdown, display
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

tf.disable_eager_execution()

2024-12-07 01:22:12.632359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733534532.644101  130415 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733534532.647628  130415 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 01:22:12.660313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/aif360/lib/python3.11/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated

In [2]:
dataset_orig = load_preproc_data_german(["age", "sex"])
privileged_groups = [{"age": 1}]
unprivileged_groups = [{"age": 0}]

/home/ubuntu/AIF360_dev/aif360/algorithms/preprocessing/optim_preproc_helpers/data_preproc_functions.py:261: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['personal_status'].replace(status_map)


In [3]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate

min_max_scaler = MaxAbsScaler()

Prep

In [4]:
dataset_orig.features = min_max_scaler.fit_transform(dataset_orig.features)

CV

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score  # Replace with desired metric
from sklearn.model_selection import KFold


def cross_validate_seeds(
    dataset, privileged_group, unprivileged_groups, seeds=5, n_folds=5
):
    """
    Runs cross-validation with multiple seeds.

    Parameters:
        model: The Scikit-learn model to validate.
        X: Feature matrix.
        y: Target vector.
        seeds: Number of seeds for cross-validation.
        folds: Number of folds for each cross-validation.

    Returns:
        results: Dictionary containing accuracy scores for each seed and fold.
    """
    results = {}

    original_df = dataset.convert_to_dataframe()[0]
    for seed in range(seeds):
        print(f"\nSeed {seed + 1}/{seeds}")
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        fold_results = []

        for fold, (train_idx, test_idx) in enumerate(kf.split(original_df)):
            print(f"{seed=} || {fold=}")
            # Splitting data
            dataset_X_train, dataset_X_test = dataset.subset(train_idx), dataset.subset(
                test_idx
            )

            # Training model
            sess = tf.Session()
            model = FairnessAdjuster(
                privileged_groups=privileged_groups,
                unprivileged_groups=unprivileged_groups,
                scope_name="debiased_classifier",
                adversary_loss_weight=0.01,
                debias=True,
                sess=sess,
                classifier_num_hidden_units=100,
            )
            model.fit(dataset_X_train)

            # Making predictions
            dataset_preds = model.predict(dataset_X_test)

            # Evaluating model
            metrics = get_metrics(
                dataset_X_test,
                dataset_preds,
                unprivileged_groups,
                privileged_groups,
            )
            fold_results.append(metrics)

            # end session
            sess.close()
            tf.reset_default_graph()
        results[f"Seed {seed}"] = fold_results

    return results

In [6]:
def get_metrics(
    dataset_orig_test, dataset_pred_test, unprivileged_groups, privileged_groups
):
    metric_dataset_debiasing_test = BinaryLabelDatasetMetric(
        dataset_orig_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
    )
    classified_metric_debiasing_test = ClassificationMetric(
        dataset_orig_test,
        dataset_pred_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups,
    )
    metrics_dict = {
        "Test Set: mean outcomes difference": metric_dataset_debiasing_test.mean_difference(),
        "Test Set: Classification accuracy": classified_metric_debiasing_test.accuracy(),
        "Test Set: Disparate impact": classified_metric_debiasing_test.disparate_impact(),
        "Test Set: Average odds difference": classified_metric_debiasing_test.average_odds_difference(),
    }
    return metrics_dict

In [7]:
results = cross_validate_seeds(
    dataset_orig,
    privileged_groups,
    unprivileged_groups,
)


Seed 1/5
seed=0 || fold=0
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2024-12-07 01:22:16.855697: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1733534536.965279  130415 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


epoch 0; iter: 0; batch classifier loss: 0.820269
epoch 1; iter: 0; batch classifier loss: 0.780574
epoch 2; iter: 0; batch classifier loss: 0.684352
epoch 3; iter: 0; batch classifier loss: 0.642363
epoch 4; iter: 0; batch classifier loss: 0.665911
epoch 5; iter: 0; batch classifier loss: 0.666919
epoch 6; iter: 0; batch classifier loss: 0.631005
epoch 7; iter: 0; batch classifier loss: 0.654215
epoch 8; iter: 0; batch classifier loss: 0.533161
epoch 9; iter: 0; batch classifier loss: 0.609605
epoch 10; iter: 0; batch classifier loss: 0.562595
epoch 11; iter: 0; batch classifier loss: 0.589132
epoch 12; iter: 0; batch classifier loss: 0.578125
epoch 13; iter: 0; batch classifier loss: 0.600686
epoch 14; iter: 0; batch classifier loss: 0.543627
epoch 15; iter: 0; batch classifier loss: 0.581816
epoch 16; iter: 0; batch classifier loss: 0.589964
epoch 17; iter: 0; batch classifier loss: 0.561705
epoch 18; iter: 0; batch classifier loss: 0.607794
epoch 19; iter: 0; batch classifier loss:

2024-12-07 01:22:18.179158: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 2; iter: 0; batch classifier loss: 0.646547
epoch 3; iter: 0; batch classifier loss: 0.637121
epoch 4; iter: 0; batch classifier loss: 0.643286
epoch 5; iter: 0; batch classifier loss: 0.593559
epoch 6; iter: 0; batch classifier loss: 0.575874
epoch 7; iter: 0; batch classifier loss: 0.647824
epoch 8; iter: 0; batch classifier loss: 0.522139
epoch 9; iter: 0; batch classifier loss: 0.590948
epoch 10; iter: 0; batch classifier loss: 0.628074
epoch 11; iter: 0; batch classifier loss: 0.572241
epoch 12; iter: 0; batch classifier loss: 0.593793
epoch 13; iter: 0; batch classifier loss: 0.601677
epoch 14; iter: 0; batch classifier loss: 0.587601
epoch 15; iter: 0; batch classifier loss: 0.497918
epoch 16; iter: 0; batch classifier loss: 0.577022
epoch 17; iter: 0; batch classifier loss: 0.534119
epoch 18; iter: 0; batch classifier loss: 0.530621
epoch 19; iter: 0; batch classifier loss: 0.540438
epoch 20; iter: 0; batch classifier loss: 0.570042
epoch 21; iter: 0; batch classifier los

...

2024-12-07 01:22:49.456997: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.663541
epoch 1; iter: 0; batch classifier loss: 0.634294
epoch 2; iter: 0; batch classifier loss: 0.595762
epoch 3; iter: 0; batch classifier loss: 0.581344
epoch 4; iter: 0; batch classifier loss: 0.550605
epoch 5; iter: 0; batch classifier loss: 0.550751
epoch 6; iter: 0; batch classifier loss: 0.591402
epoch 7; iter: 0; batch classifier loss: 0.552181
epoch 8; iter: 0; batch classifier loss: 0.547054
epoch 9; iter: 0; batch classifier loss: 0.592685
epoch 10; iter: 0; batch classifier loss: 0.552673
epoch 11; iter: 0; batch classifier loss: 0.525316
epoch 12; iter: 0; batch classifier loss: 0.537887
epoch 13; iter: 0; batch classifier loss: 0.579048
epoch 14; iter: 0; batch classifier loss: 0.554685
epoch 15; iter: 0; batch classifier loss: 0.563186
epoch 16; iter: 0; batch classifier loss: 0.584138
epoch 17; iter: 0; batch classifier loss: 0.556098
epoch 18; iter: 0; batch classifier loss: 0.589743
epoch 19; iter: 0; batch classifier loss:

2024-12-07 01:22:50.870638: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.735410
epoch 1; iter: 0; batch classifier loss: 0.683140
epoch 2; iter: 0; batch classifier loss: 0.657581
epoch 3; iter: 0; batch classifier loss: 0.583051
epoch 4; iter: 0; batch classifier loss: 0.603309
epoch 5; iter: 0; batch classifier loss: 0.600414
epoch 6; iter: 0; batch classifier loss: 0.562279
epoch 7; iter: 0; batch classifier loss: 0.605130
epoch 8; iter: 0; batch classifier loss: 0.569584
epoch 9; iter: 0; batch classifier loss: 0.594956
epoch 10; iter: 0; batch classifier loss: 0.610973
epoch 11; iter: 0; batch classifier loss: 0.549844
epoch 12; iter: 0; batch classifier loss: 0.589856
epoch 13; iter: 0; batch classifier loss: 0.573770
epoch 14; iter: 0; batch classifier loss: 0.572699
epoch 15; iter: 0; batch classifier loss: 0.560102
epoch 16; iter: 0; batch classifier loss: 0.572233
epoch 17; iter: 0; batch classifier loss: 0.633607
epoch 18; iter: 0; batch classifier loss: 0.624999
epoch 19; iter: 0; batch classifier loss:

2024-12-07 01:22:52.309426: W tensorflow/c/c_api.cc:305] Operation '{name:'debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign' id:265 op device:{requested: '', assigned: ''} def:{{{node debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1, debiased_classifier/debiased_classifier/classifier_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [22]:
agg = {}
for seed, res_dict in results.items():
    agg[seed] = pd.DataFrame(res_dict).mean()

pd.DataFrame.from_dict(agg).T

,Test Set: mean outcomes difference,Test Set: Classification accuracy,Test Set: Disparate impact,Test Set: Average odds difference
Seed 0,-0.145631,0.701,0.793761,-0.224643
Seed 1,-0.155718,0.711,0.741260,-0.279001
Seed 2,-0.152968,0.705,0.771896,-0.232746
Seed 3,-0.146748,0.709,0.788861,-0.233485
Seed 4,-0.154603,0.708,0.693850,-0.324598
